<a href="https://colab.research.google.com/github/John-Spenceley/AI-Basics/blob/main/Building_an_LLM_(Basics).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**How to Build an LLM Playground (In Detail)**

**Purpose**: Create a hands-on, code-light path to understanding how Large Language Models work—so non-coders can use, evaluate, and eventually tailor an LLM without getting bogged down in engineering details.

**Who This Is For:**



*   Non-coders / beginners who have a rudimentary understanding of Python
*   Product, research, or ops folks who want to reason about LLM behavior
*   Makers planning a simple, focused LLM for real users

**Required Tools:**

*   Google Colab (https://colab.research.google.com/)
*   Optional local run with Jupyter/VS Code


**Learning Objectives**

1. **Tokenization** — Turning Text into Tokens

*   Understand how raw text is split into a sequence of discrete tokens (the building blocks of LLMs).
*   Visualize how punctuation, emojis, and word fragments are represented.
*   Learn why tokenization affects both cost and creativity in text generation.


2. **Inspecting GPT-2 & Transformer Architecture**

*   Explore the core building blocks of LLMs: embeddings, self-attention, and layers.
*   See how GPT-2 represents the broader class of Transformer-based models that power today’s AI systems.
*   Understand at a conceptual level how models “predict the next token” to form language.



3. **Loading Pre-Trained LLMs (Using Hugging Face)**

*   Learn how to load existing pre-trained models in one line of code using the Hugging Face library.
*   No training required—just loading, prompting, and observing.
*   Gain confidence in exploring different model types (GPT-2, Qwen, etc.).

4. **Decoding Strategies — How Models Generate Text**

*   Experiment with decoding parameters: temperature, top-k, and top-p.
*  Understand how these affect creativity, coherence, and factuality.

*   Compare deterministic (greedy) vs. probabilistic sampling.

5. **Completion vs. Instruction Fine-Tuned Models**

*   Learn the difference between completion models (predict the next word) and instruction-tuned models (follow directions).

*   Understand why instruction tuning makes models like ChatGPT easier for everyday users—especially non-coders.
*   Practice prompting both types and see how their behaviors differ.

**By the End, You’ll Be Able To:**

*   Explain what tokenization means and why it matters.
*   Describe the basic structure of a Transformer model.
*   Load and interact with pre-trained models confidently.
*   Adjust decoding strategies to control style and randomness.
*   Differentiate between completion and instruction-tuned LLMs—knowing which is better for non-coder projects.


##**Setup Cell — Import and Version Check**

1. **Purpose**
   * Ensure that all required LLM libraries are installed and correctly loaded in your Colab environment.
   * Confirm compatibility between the deep-learning framework (PyTorch), the model library (Transformers), and the tokenizer library (TikToken).
   * This verification helps prevent runtime errors caused by version mismatches.

2. **Libraries Used**
   * **torch** — Core deep learning framework (handles tensors, GPU computation, and neural network training).
   * **transformers** — Hugging Face library providing access to pre-trained LLMs like GPT-2, BERT, and Qwen.
   * **tiktoken** — OpenAI’s fast tokenizer that converts text into tokens (numbers) and back.

3. **What Happens**
   * The libraries are imported.
   * The versions of `torch` and `transformers` are printed to verify installation.
   * This acts as a quick diagnostic step before loading or running any model.


In [1]:
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)

torch 2.8.0+cu126 | transformers 4.57.0


## Tokenization — Turning Text into Tokens

A neural network can’t digest raw text — it needs numbers.  
Tokenization is the process of converting text into integer IDs that a model can understand.

In this section, you'll learn how tokenization is implemented in practice.

### Tokenization Methods
Tokenization methods generally fall into three main categories:

1. **Word-level tokenization** — Split text by spaces; each word becomes a token.  
2. **Character-level tokenization** — Each character (letter, punctuation, emoji) becomes a token.  
3. **Subword-level tokenization** — Breaks words into smaller pieces for efficiency and flexibility (used by GPT-2 and most modern LLMs).

---

### 1.1 – Word-Level Tokenization
Split text on whitespace and store each word as a token.

In [2]:
# 1. Tiny corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# 2. Build the vocabulary
PAD, UNK = "[PAD]", "[UNK]"
words = set()
for doc in corpus:
    words.update(doc.lower().split())

vocab = [PAD, UNK] + sorted(words)
word2id = {w: i for i, w in enumerate(vocab)}
id2word = {i: w for w, i in word2id.items()}

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])

# 3. Encode / decode functions
def encode(text):
    return [word2id.get(w, word2id[UNK]) for w in text.lower().split()]

def decode(ids):
    return " ".join(id2word[i] for i in ids if i != word2id[PAD])

# 4. Demo
sample = "The brown unicorn jumps"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Vocabulary size: 21 words
First 15 vocab entries: ['[PAD]', '[UNK]', 'brown', 'converts', 'dog', 'fox', 'jumps', 'language', 'large', 'lazy', 'models', 'next', 'numbers', 'over', 'predict']

Input text : The brown unicorn jumps
Token IDs  : [17, 2, 1, 6]
Decoded    : the brown [UNK] jumps


**Understanding the Output**

1. **Vocabulary size: 21 words**  
   The model found 21 unique tokens (words) in the sample text collection, plus two special tokens — `[PAD]` and `[UNK]`.  
   These form the *vocabulary*, which is the list of all known words.

2. **First 15 vocab entries:**  
   `['[PAD]', '[UNK]', 'brown', 'converts', 'dog', 'fox', 'jumps', 'language', 'large', 'lazy', 'models', 'next', 'numbers', 'over', 'predict']`  
   This shows the first 15 tokens in alphabetical order.  
   *`[PAD]`* is used for padding shorter sequences, and *`[UNK]`* represents unknown words.

3. **Input text : The brown unicorn jumps**  
   This is the example sentence being tokenized.

4. **Token IDs  : [17, 2, 1, 6]**  
   Each number corresponds to the position of a word in the vocabulary:  
   - `17` = "the"  
   - `2`  = "brown"  
   - `1`  = `[UNK]` (unknown word, because “unicorn” isn’t in the vocabulary)  
   - `6`  = "jumps"

5. **Decoded : the brown [UNK] jumps**  
   The tokens are converted back into words.  
   “Unicorn” was replaced with `[UNK]` because it wasn’t in the vocabulary.

**In summary:**  
This output demonstrates how tokenization turns words into numerical IDs, and how unknown words are handled when they don’t exist in the training vocabulary.


## 1.2 – Character-Level Tokenization

Every single character (including spaces, punctuation, and emojis) gets its own ID.  
This guarantees zero out-of-vocabulary (OOV) issues but produces much longer sequences.

Character-level tokenization is useful when dealing with small vocabularies or languages with many rare words, but it’s computationally heavier since each word is broken into multiple characters.


In [3]:
# 1. Build a fixed vocabulary
import string

letters = list(string.ascii_lowercase + string.ascii_uppercase)  # a–z + A–Z
special = ["[PAD]", "[UNK]"]  # padding + unknown
vocab = special + letters

char2id = {ch: idx for idx, ch in enumerate(vocab)}
id2char = {idx: ch for ch, idx in char2id.items()}

print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")

# 2. Encode / decode
def encode(text):
    """Convert text → list of IDs (unknown chars → [UNK])."""
    unk_id = char2id["[UNK]"]
    return [char2id.get(ch, unk_id) for ch in text]

def decode(ids):
    """Convert list of IDs back to characters."""
    return "".join(id2char[i] for i in ids if i != char2id["[PAD]"])

# 3. Demo
sample = "Hello"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Vocabulary size: 54 (52 letters + 2 specials)

Input text : Hello
Token IDs  : [35, 6, 13, 13, 16]
Decoded    : Hello


**Understanding the Output**

1. **Vocabulary size: 54 (52 letters + 2 specials)**  
   There are 26 lowercase and 26 uppercase letters, plus two special tokens:  
   `[PAD]` for padding sequences, and `[UNK]` for unknown characters.

2. **Input text : Hello**  
   The string you want to tokenize.

3. **Token IDs :**  
   Each character in “Hello” is converted to its numeric ID using the `char2id` dictionary.  
   For example, `H`, `e`, `l`, and `o` each have their own assigned number.

4. **Decoded : Hello**  
   The token IDs are converted back to characters using the `id2char` mapping, reconstructing the original text.

**In summary:**  
This demonstrates how character-level tokenization works.  
Each character (including case differences) becomes its own token, eliminating OOV errors but resulting in longer token sequences.


## 1.3 – Subword-Level Tokenization

Subword methods such as **Byte-Pair Encoding (BPE)**, **WordPiece**, and **SentencePiece** learn the most common character combinations and group them into tokens.  

For example, the word *unbelievable* might be split into three tokens:  
`["un", "believ", "able"]`

This approach strikes a balance between word-level and character-level methods, solving their main limitations — it handles unknown words efficiently while keeping sequence lengths manageable.

---

### How BPE Works

1. Start with individual characters (bytes) — each is its own token.  
2. Count all adjacent pairs of tokens in a large corpus.  
3. Merge the most frequent pair into a new token.  
4. Repeat steps 2–3 until you reach the target vocabulary size (e.g., 50 000).  

Let’s see **BPE** in practice using GPT-2’s pretrained tokenizer.

In [4]:
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE)
from transformers import AutoTokenizer

bpe_tok = AutoTokenizer.from_pretrained("gpt2")

print("Vocab size:", bpe_tok.vocab_size)
print("Special tokens:", bpe_tok.all_special_tokens)

# 2. Encode / decode
def encode(text):
    return bpe_tok.encode(text)

def decode(ids):
    return bpe_tok.decode(ids)

# 3. Demo
sample = "Unbelievable tokenization powers! 🚀"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Tokens     :", bpe_tok.convert_ids_to_tokens(ids))
print("Decoded    :", recovered)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Vocab size: 50257
Special tokens: ['<|endoftext|>']

Input text : Unbelievable tokenization powers! 🚀
Token IDs  : [3118, 6667, 11203, 540, 11241, 1634, 5635, 0, 12520, 248, 222]
Tokens     : ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', '!', 'ĠðŁ', 'ļ', 'Ģ']
Decoded    : Unbelievable tokenization powers! 🚀


**Understanding the Output**

1. **Vocab size**  
   Shows how many unique subword tokens exist in GPT-2’s vocabulary (about 50 000).

2. **Special tokens**  
   Lists tokens like `` or `` that mark text boundaries or padding.

3. **Token IDs**  
   Each subword in the input is converted into its corresponding numerical ID.

4. **Tokens**  
   Displays the actual subword pieces created by BPE — for example:  
   `["Un", "believ", "able", " token", "ization", " powers", "!", "Ġ🚀"]`

5. **Decoded text**  
   Converts the token IDs back into readable text, confirming that the tokenizer can perfectly reconstruct the original input.

**In summary:**  
Subword-level tokenization allows language models to handle both familiar and unseen words efficiently by combining the flexibility of character-level methods with the compactness of word-level ones.


## 1.4 – TikToken

`tiktoken` is a production-ready, high-speed tokenization library used by OpenAI models.  
It’s optimized for performance and supports the same tokenization rules used in GPT-3, GPT-3.5, and GPT-4.

In this section, we’ll compare the **older GPT-2 encoding** (`gpt2`) with the **newer GPT-4 encoding** (`cl100k_base`).

The newer encoding supports a much larger vocabulary and handles emojis, punctuation, and multilingual text more efficiently.


In [5]:
# Compare GPT-2 and GPT-4 encodings using TikToken
import tiktoken

encodings = [
    ("gpt2", tiktoken.get_encoding("gpt2")),
    ("cl100k_base", tiktoken.get_encoding("cl100k_base")),
]

sentence = "The 🌟 star-player scored 40 points!"

for name, enc in encodings:
    print(f"\n=== {name} ===")
    print("Vocabulary size:", enc.n_vocab)

    # Encode the sample sentence
    ids = enc.encode(sentence)
    tokens = [enc.decode([i]) for i in ids]
    print(f"Sentence splits into {len(ids)} tokens:")
    print(list(zip(tokens, ids)))

    # Show a few arbitrary token→ID examples from the vocab
    some_ids = [0, 1, 2, 198, 50256]
    print("Sample tokens from the vocabulary:")
    print([(enc.decode([i]), i) for i in some_ids])



=== gpt2 ===
Vocabulary size: 50257
Sentence splits into 11 tokens:
[('The', 464), (' �', 12520), ('�', 234), ('�', 253), (' star', 3491), ('-', 12), ('player', 7829), (' scored', 7781), (' 40', 2319), (' points', 2173), ('!', 0)]
Sample tokens from the vocabulary:
[('!', 0), ('"', 1), ('#', 2), ('\n', 198), ('<|endoftext|>', 50256)]

=== cl100k_base ===
Vocabulary size: 100277
Sentence splits into 11 tokens:
[('The', 791), (' �', 11410), ('�', 234), ('�', 253), (' star', 6917), ('-player', 43467), (' scored', 16957), (' ', 220), ('40', 1272), (' points', 3585), ('!', 0)]
Sample tokens from the vocabulary:
[('!', 0), ('"', 1), ('#', 2), ('\n', 198), ('parable', 50256)]


**Understanding the Output**

1. **Vocabulary size**  
   Displays how many tokens exist in each encoding.  
   - `gpt2` uses around 50,000 tokens.  
   - `cl100k_base` (used by GPT-4) supports over 100,000 tokens.

2. **Sentence splits into ... tokens**  
   Shows how the same sentence is divided differently depending on the encoding.  
   GPT-4’s `cl100k_base` tends to use fewer tokens for the same text because it has a richer vocabulary.

3. **Token–ID pairs**  
   Each token (word, subword, or emoji) is paired with its corresponding integer ID — how models see text internally.

4. **Sample tokens from the vocabulary**  
   Prints a few tokens and their IDs to illustrate how special tokens or punctuation are represented.

**In summary:**  
`tiktoken` provides the exact tokenizer used by OpenAI models, allowing you to measure, visualize, and understand how text is split into tokens.  
It’s especially useful when estimating token costs or preparing text for OpenAI API inputs.


## Tokenization 1.5 – Key Takeaways

**Word-level:**  
Simple to implement but brittle — struggles with *out-of-vocabulary (OOV)* words that were not seen during training.

**Character-level:**  
Handles every possible input, but produces long token sequences and is less efficient for training large models.

**Subword-level (BPE / Byte-Level BPE):**  
Strikes a balance between the two — compact, efficient, and capable of representing new words through smaller subword units.  
Used by most modern language models (e.g., GPT-2, GPT-3, GPT-4, BERT).

**TikToken:**  
Demonstrates how production-grade models tokenize using optimized, pre-trained subword vocabularies.  
It provides the same fast and memory-efficient tokenization used inside OpenAI’s GPT models.


## 2 – What is a Language Model?

At its core, a **language model (LM)** is a large mathematical function built from many neural-network layers.  
Given a sequence of tokens [t₁, t₂, …, tₙ], it learns to output a probability for the next token tₙ₊₁.

Each layer performs simple operations (matrix multiplication, attention, etc.).  
Stacking hundreds of these layers allows the model to capture patterns and relationships in text.

The final output is a vector of scores representing how likely each possible next token is.  
You can think of the entire network as one enormous equation whose parameters were tuned during training to minimize prediction error.


### 2.1 – A Single Linear Layer
Before exploring the Transformer, let’s start with the simplest building block.

A **Linear layer** performs the operation *y = Wx + b*  
where  
- **x** is the input vector  
- **W** is the learned weight matrix  
- **b** is the learned bias vector  

Chaining many such linear layers (with nonlinear activations in between) gives neural networks their expressive power.


In [ ]:
import torch
import torch.nn as nn

# Define a simple Linear layer manually
class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        return torch.matmul(x, self.weight.t()) + self.bias

lin = Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


In [ ]:
# Same operation using PyTorch’s built-in Linear layer
lin = nn.Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


**Explanation**  
The Linear layer multiplies the input vector by a learned weight matrix and adds a bias vector.  
This transforms the input into a new representation — a basic step repeated thousands of times inside LLMs.


### 2.2 – A Transformer Layer
Most LLMs are built as a stack of identical **Transformer blocks**, each containing two main parts:

| Step | What it does | Where it lives in code |
|:--|:--|:--|
| Multi-Head Self-Attention | Each token looks at other tokens to decide what matters | `block.attn` |
| Feed-Forward Network (MLP) | Re-mixes information token by token | `block.mlp` |

Below we load the smallest public GPT-2 (124 M parameters), grab its first block, and inspect its modules.


In [6]:
import torch
from transformers import GPT2LMHeadModel

# Load GPT-2 (124 M parameters)
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
block = gpt2.transformer.h[0]   # GPT-2 has 12 such layers

for name, module in block.named_children():
    print(f"{name:7s} → {module.__class__.__name__}")

print("\n=== First Transformer Block ===")
print(block, "\n")

# Run a tiny forward pass through one block
seq_len = 8
dummy_tokens = torch.randint(0, gpt2.config.vocab_size, (1, seq_len))

with torch.no_grad():
    hidden = (
        gpt2.transformer.wte(dummy_tokens) +
        gpt2.transformer.wpe(torch.arange(seq_len))
    )
    out = block(hidden, layer_past=None, use_cache=False)[0]

print("\nOutput shape :", out.shape)   # (batch, seq_len, hidden_size)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

ln_1    → LayerNorm
attn    → GPT2Attention
ln_2    → LayerNorm
mlp     → GPT2MLP

=== First Transformer Block ===
GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
) 


Output shape : torch.Size([1, 8, 768])


**Explanation**  
A Transformer block contains an **attention mechanism** and a **feed-forward network**.  
Each token gathers information from others, enabling the model to capture long-range relationships.


### 2.3 – Inside GPT-2
GPT-2 is just many of those Transformer blocks arranged sequentially.  
Let’s print the modules inside the Transformer stack.


In [ ]:
for name, module in gpt2.transformer.named_children():
    print(f"{name:7s} → {module.__class__.__name__}")


**Summary of Main Modules**

| Step | What it does | Why it matters |
|:--|:--|:--|
| Token → Embedding | Converts token IDs into vectors | Gives the model numeric handles on words |
| Positional Encoding | Adds information about word order | Order matters in language |
| Multi-Head Self-Attention | Each token asks “which other tokens should I attend to?” | Captures context and relationships |
| Feed-Forward Network | Two Linear layers with a non-linearity | Adds depth and pattern mixing |
| LayerNorm & Residual | Stabilize training and help gradients flow | Keep deep models trainable |


### 2.4 – LLM Output
Passing a token sequence through an LLM produces a tensor of **logits** with shape  
*(batch_size, seq_len, vocab_size)*.  

Applying `softmax` on the last dimension converts these logits into probabilities for each possible next token.


In [7]:
import torch
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load GPT-2 and tokenizer if needed
try:
    gpt2
except NameError:
    gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Tokenize input
text = "Hello my name"
input_ids = tokenizer(text, return_tensors="pt").input_ids   # (1, seq_len)

with torch.no_grad():
    logits = gpt2(input_ids).logits                         # (1, seq_len, vocab_size)

print("Logits shape :", logits.shape)

# Predict next token
probs = F.softmax(logits[0, -1], dim=-1)
topk = torch.topk(probs, 5)

print("\nTop-5 predictions for the next token:")
for idx, p in zip(topk.indices.tolist(), topk.values.tolist()):
    print(f"{tokenizer.decode([idx]):>10s} — {p:.4f}")


Logits shape : torch.Size([1, 3, 50257])

Top-5 predictions for the next token:
        is — 0.7773
         , — 0.0373
        's — 0.0332
       was — 0.0127
       and — 0.0076


**Explanation**  
The output logits represent scores for every token in the vocabulary.  
After applying softmax, we get probabilities for the next token.  
Printing the top-5 tokens shows which words the model believes are most likely to follow the input.


### 2.5 – Key Takeaway

A language model is not mystical — it’s a large composition of simple, understandable layers trained to predict the next token in a sequence.

By stacking Linear layers, attention mechanisms, and normalization steps at scale,  
modern LLMs can capture grammar, context, and meaning from text data through pure pattern prediction.


## 3 – Generation

Once a language model is trained to predict the probabilities of the next token, we can **generate text** from it.  
This process is called **decoding** or **sampling**.

At each step, the model outputs a probability distribution over the next token.  
The decoding algorithm decides which token to pick next, then continues predicting subsequent tokens.

---

### Common Decoding Strategies

| Strategy | Description | Behavior |
|:--|:--|:--|
| **Greedy** | Always pick the single most probable next token | Deterministic, but can become repetitive |
| **Top-k Sampling** | Randomly sample from the top-k most likely tokens | Adds variety while staying coherent |
| **Nucleus (Top-p)** | Sample from the smallest set of tokens whose probabilities sum to *p* | Adapts dynamically to context |
| **Beam Search** | Keeps multiple candidate sequences and expands the best ones | More structured, often used in translation |
| **Temperature** | A “creativity knob”: higher values flatten the probability distribution | Lower = precise / Higher = diverse |

---


In [ ]:
# 3.1 – Greedy Decoding
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODELS = {"gpt2": "gpt2"}
tokenizers, models = {}, {}

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load GPT-2
for key, mid in MODELS.items():
    tok = AutoTokenizer.from_pretrained(mid)
    mdl = AutoModelForCausalLM.from_pretrained(mid).eval().to(device)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    mdl.config.pad_token_id = tok.pad_token_id
    tokenizers[key], models[key] = tok, mdl
    print(f"Loaded {mid} as {key}")

# Generation function
def generate(model_key, prompt, strategy="greedy", max_new_tokens=100):
    tok, mdl = tokenizers[model_key], models[model_key]
    enc = tok(prompt, return_tensors="pt").to(mdl.device)
    gen_args = dict(**enc, max_new_tokens=max_new_tokens, pad_token_id=tok.pad_token_id)

    if strategy == "greedy":
        gen_args["do_sample"] = False
    elif strategy == "top_k":
        gen_args.update(dict(do_sample=True, top_k=50, temperature=0.9))
    elif strategy == "top_p":
        gen_args.update(dict(do_sample=True, top_p=0.9, temperature=0.9))

    out = mdl.generate(**gen_args)
    return tok.decode(out[0], skip_special_tokens=True)

# Demo: Greedy decoding
tests = ["Once upon a time", "What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Greedy ==")
    print(generate("gpt2", prompt, "greedy", 80))


**Explanation**

Greedy decoding always selects the highest-probability token at every step.  
It’s efficient but can easily fall into repetition (e.g., “The cat is is is…”) and may miss more interesting continuations with slightly lower initial probability.


### 3.2 – Top-k and Top-p Sampling
These methods introduce randomness for more natural and creative outputs.

* **Top-k Sampling:** randomly sample from the top *k* most likely tokens.  
* **Top-p (Nucleus) Sampling:** dynamically choose the smallest subset of tokens whose cumulative probability ≥ *p*.


In [ ]:
# Compare Top-p Sampling
tests = ["Once upon a time", "What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p ==")
    print(generate("gpt2", prompt, "top_p", 40))


**Explanation**

Top-p sampling (with p ≈ 0.9) usually produces smoother, more human-like text.  
Because it samples from a variable-sized pool of likely tokens, it balances creativity and coherence better than greedy decoding.


### 3.3 – Try It Yourself

Scroll to the list called `tests` above and modify it to include your own prompts.

You can also experiment with these parameters:

| Parameter | Description | Typical Range |
|:--|:--|:--|
| `strategy` | `"greedy"`, `"top_k"`, `"top_p"`, `"beam"` | — |
| `temperature` | Controls randomness | 0.2 – 2.0 |
| `top_k` | Number of tokens considered in top-k sampling | 10 – 100 |
| `top_p` | Cumulative probability cutoff for top-p sampling | 0.8 – 0.95 |

**Tip:**  
Try generating the same prompt with both **greedy** and **top-p = 0.9** — notice how tone, vocabulary, and rhythm change with temperature adjustments.


## 4 – Completion vs. Instruction-Tuned LLMs

So far, we’ve seen that we can use **GPT-2** to generate text continuations.  
However, GPT-2 is a *completion model*: it simply continues the given text, without understanding it as a question or request.

**Instruction-tuned LLMs** (like Qwen-Chat, ChatGPT, or Llama-2-Chat) go through an additional training stage called **post-training**.  
This stage teaches them to interpret prompts as instructions rather than raw text.

---

### Key Differences

| Model Type | Behavior | Training Focus |
|:--|:--|:--|
| **Completion Model** (e.g., GPT-2) | Continues text in the same style | Predicts next tokens only |
| **Instruction-Tuned Model** (e.g., Qwen-Chat) | Reads prompts as requests and responds helpfully | Fine-tuned with human feedback and dialogue data |

---

### Why Instruction-Tuned Models Feel “Smarter”

Because of **post-training**, instruction-tuned models will:
- Read the entire prompt as a *request*, not just as text to mimic  
- Stay in dialogue mode, answering questions or following instructions  
- Refuse unsafe or disallowed prompts  
- Maintain a consistent persona (“Assistant”) rather than drifting into storytelling  


In [ ]:
# 4.1 – Qwen1.5-Chat vs. GPT-2

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}

tokenizers, models = {}, {}
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load both GPT-2 and Qwen-Chat
for key, mid in MODELS.items():
    tok = AutoTokenizer.from_pretrained(mid)
    mdl = AutoModelForCausalLM.from_pretrained(mid).eval().to(device)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    mdl.config.pad_token_id = tok.pad_token_id
    tokenizers[key], models[key] = tok, mdl
    print(f"Loaded {mid} as {key}")


**Note:**  
This downloads two small models:  
- **GPT-2 (124 M parameters)** — a base *completion* model  
- **Qwen-1.5-Chat (1.8 B parameters)** — an *instruction-tuned* chat model  

The download may take a few minutes the first time; subsequent runs use cached models.


In [ ]:
# Compare GPT-2 vs. Qwen-Chat on identical prompts

tests = [
    ("Once upon a time", "greedy"),
    ("What is 2+2?", "top_k"),
    ("Suggest a party theme.", "top_p")
]

for prompt, strategy in tests:
    for key in ["gpt2", "qwen"]:
        print(f"\n== {key.upper()} | {strategy} ==")
        print(generate(key, prompt, strategy, 80))


**Understanding the Comparison**

1. **GPT-2 Output**  
   - Treats the prompt as story text.  
   - Produces narrative or associative continuations.  
   - Doesn’t recognize commands or questions explicitly.

2. **Qwen-Chat Output**  
   - Interprets the prompt as a *task* or *question*.  
   - Provides concise, direct answers.  
   - Maintains an assistant-style tone.

**In summary:**  
Instruction-tuned models extend base LLMs by aligning them with *human intentions*, making them interactive, helpful, and safe for real-world use.


## 5 – A Small LLM Playground (Optional)

This optional section builds a **mini interactive playground** where you can:
- Enter a text prompt  
- Choose a model (GPT-2 or Qwen-Chat)  
- Select a decoding strategy (greedy, top-k, or top-p)  
- Adjust the temperature to control creativity  

Press **Generate** to watch the model respond in real time.


In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}

tokenizers, models = {}, {}
device = "cuda" if torch.cuda.is_available() else "cpu"

for key, mid in MODELS.items():
    tok = AutoTokenizer.from_pretrained(mid)
    mdl = AutoModelForCausalLM.from_pretrained(mid).eval().to(device)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    mdl.config.pad_token_id = tok.pad_token_id
    tokenizers[key], models[key] = tok, mdl
    print(f"Loaded {mid} as {key}")


import ipywidgets as widgets
from IPython.display import display, Markdown

# Make sure models and tokenizers are loaded
try:
    tokenizers
    models
except NameError:
    raise RuntimeError("Please run the earlier setup cells that load the models before using the playground.")

# ---------------------------------------------------------
# Text Generation Function
# ---------------------------------------------------------
def generate_playground(model_key, prompt, strategy="greedy", temperature=1.0, max_new_tokens=100):
    tok, mdl = tokenizers[model_key], models[model_key]
    enc = tok(prompt, return_tensors="pt").to(mdl.device)
    gen_args = dict(**enc, max_new_tokens=max_new_tokens, pad_token_id=tok.pad_token_id)

    if strategy == "greedy":
        gen_args["do_sample"] = False
    elif strategy == "top_k":
        gen_args.update(dict(do_sample=True, top_k=50, temperature=temperature))
    elif strategy == "top_p":
        gen_args.update(dict(do_sample=True, top_p=0.9, temperature=temperature))
    else:
        raise ValueError("Unknown strategy")

    out = mdl.generate(**gen_args)
    return tok.decode(out[0], skip_special_tokens=True)

# ---------------------------------------------------------
# Build Interactive UI
# ---------------------------------------------------------

# Text box for the user prompt
prompt_box = widgets.Textarea(
    value="Tell me a fun fact about space.",
    placeholder="Type your prompt here",
    description="Prompt:",
    layout=widgets.Layout(width="100%", height="120px")
)

# Dropdown for model selection
model_dropdown = widgets.Dropdown(
    options=[("GPT-2", "gpt2"), ("Qwen-1.5-Chat", "qwen")],
    value="gpt2",
    description="Model:"
)

# Dropdown for decoding strategy
strategy_dropdown = widgets.Dropdown(
    options=[("Greedy", "greedy"), ("Top-k", "top_k"), ("Top-p", "top_p")],
    value="greedy",
    description="Strategy:"
)

# Temperature slider
temperature_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=2.0,
    step=0.1,
    description="Temp:"
)

# Generate button
generate_button = widgets.Button(description="Generate", button_style="primary")

# Output area
output_area = widgets.Output()

# ---------------------------------------------------------
# Define button callback
# ---------------------------------------------------------
def on_generate(_):
    output_area.clear_output()
    with output_area:
        try:
            result = generate_playground(
                model_dropdown.value,
                prompt_box.value,
                strategy_dropdown.value,
                temperature_slider.value
            )
            display(Markdown(f"**Output:**\n\n{result}"))
        except Exception as e:
            print("Error:", e)

# Attach callback to button
generate_button.on_click(on_generate)

# ---------------------------------------------------------
# Layout and Display
# ---------------------------------------------------------
ui = widgets.VBox([
    prompt_box,
    widgets.HBox([model_dropdown, strategy_dropdown, temperature_slider]),
    generate_button,
    output_area
])

display(ui)


Loaded gpt2 as gpt2


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

Loaded Qwen/Qwen1.5-1.8B-Chat as qwen


**How to Use**

1. Type your own prompt in the text box.  
2. Choose a model and decoding strategy.  
3. Adjust the *temperature* (lower = precise / higher = creative).  
4. Press **Generate** to see the model’s output below.  

**Tip:**  
Try the same prompt with both GPT-2 and Qwen-Chat — notice how GPT-2 continues text, while Qwen-Chat interprets your request and responds conversationally.
